In [1]:
import tweepy 
import json
import pandas as pd
import csv

In [2]:
consumer_key = "mUYnzviCKjVGmoKrtxL7x0sea" 
consumer_secret = "ozOCWhjPx2AvQXTEcWuB82Q3fMYxD4CvFqkfe1NLYUyTjyNkEy"
access_key = "400660712-Lrk7Utlnxhz6VUDaLf5T3ath3MdM8NVPyNF7onQn"
access_secret = "zJuVIiSs9Dgg5Kemlvre8cKpjejPFuf6o8SZhF8Ica4tx"

In [3]:
# Authorization to consumer key and consumer secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# Access to user's access key and access secret 
auth.set_access_token(access_key, access_secret) 
  
# Calling api 
api = tweepy.API(auth) 

In [18]:
def fetch_tweets( handle, filename ) :
    tweets_data = []
    header = [ "tweet_id", "tweet_text", "content-type", "url", "hashtags", "likes", "retweet_count", "user_mentions" ] 
    req_count = 3200
    last_id = ""

    while req_count > 0 :

        if req_count < 200 :
            ct = req_count 
        else :
            ct = 200

        if req_count <= 3200 :
            tweets = api.user_timeline( screen_name = handle, count = ct, tweet_mode = "extended" )
        else :
            tweets = api.user_timeline( screen_name = handle, since_id = last_id, count = ct, tweet_mode = "extended" )
        if len( tweets ) > 0 :
            for i in range( len( tweets ) ) :
                tweet_data = []
                tweet_data.append( tweets[ i ]._json[ "id"] )
                tweet_data.append( str( tweets[ i ]._json[ "full_text"] ).replace( ";", "" ) )

                if 'media' in tweets[ i ]._json[ "entities" ]:
                    tweet_data.append(tweets[ i ]._json[ "entities" ][ "media" ][0][ "type" ])
                    tweet_data.append(tweets[ i ]._json[ "entities" ][ "media" ][0][ "url" ])
                else:
                    tweet_data.append("No")
                    tweet_data.append("No")
                if len( tweets[ i ]._json[ "entities" ][ "hashtags" ] ) > 0 :
                    tweet_data.append( tweets[ i ]._json[ "entities" ][ "hashtags" ][ 0 ][ "text" ]  )
                else :
                    tweet_data.append( "" )
                if not str( tweets[ i ]._json[ "favorite_count" ] ).isnumeric() :
                    print( tweets[ i ]._json[ "favorite_count" ] )
                else:
                    tweet_data.append( tweets[ i ]._json[ "favorite_count" ] )
                tweet_data.append( tweets[ i ]._json[ "retweet_count" ] )
                tweet_data.append( tweets[ i ]._json[ "entities" ][ "user_mentions" ] )
                tweets_data.append( tweet_data )
                last_id = tweets[ i ]._json[ "id"]
        else :
            raise Exception("Limit exceeded")

        req_count = req_count - len( tweets )

    tweets_data = pd.DataFrame( tweets_data, columns = header )
    tweets_data.to_csv( filename )

In [27]:
fetch_tweets( "INCindia", "congress_twitter2.csv")

In [19]:
fetch_tweets( "bjp4india", "bjp_twitter1.csv")